<a href="https://colab.research.google.com/github/Murad0998/statistics-project/blob/hypotheses/Categorial_Hypotheses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

In [ ]:
df = pd.read_csv('bank_clean.csv')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,y_bin
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0


В рамках EDA мы изучили связь признаков с целевой переменной y_bin (0 — клиент не подписал депозит, 1 — подписал). Для категориальных признаков (например, job) мы построили график доли подписок по категориям:

На графике «Доля y=1 по профессиям» видно, что выборочные доли подписки заметно различаются между профессиями (у одних категорий доля y=1 существенно выше, у других — ниже). Однако такие различия могут быть обусловлены случайными колебаниями выборки и различными размерами групп. Поэтому далее мы переходим от визуального наблюдения к формальной проверке статистической значимости связи между признаками.

Так как job — категориальная переменная с несколькими уровнями, а y_bin — бинарная категориальная переменная, корректным способом проверить наличие зависимости является χ²-тест независимости на таблице сопряженности job × y_bin. Данный тест отвечает на вопрос: отличается ли распределение y_bin по категориям job настолько, что это нельзя объяснить случайностью при условии независимости.

В EDA видны различия конверсии по профессиям. Если связь реальная, маркетинг/колл-центр могут по-разному сегментировать предложения и сценарии звонка, повышая конверсию при тех же затратах.

Гипотезы:
- $H_0$: профессия клиента не связана с решением подписаться на депозит (распределение y_bin одинаково для всех job).
- $H_1$: профессия клиента связана с решением подписаться (распределения различаются).

Уровень значимости: $\alpha$ = 0.05

In [ ]:
d1 = df.sample(n=300, random_state=42)
d2 = df.sample(n=2500, random_state=42)

def cv(tbl):
    cs = chi2_contingency(tbl, correction=False)[0]
    s = tbl.sum().sum()
    p2 = cs / s
    r, k = tbl.shape
    p2c = max(0, p2 - ((k-1)*(r-1))/(s-1))
    rc = r - ((r-1)**2)/(s-1)
    kc = k - ((k-1)**2)/(s-1)
    v = np.sqrt(p2c / min((kc-1), (rc-1)))
    return v

def icv(v):
    if v < 0.1:
        return "очень слабая"
    elif v < 0.3:
        return "слабая"
    elif v < 0.5:
        return "умеренная"
    else:
        return "сильная"

print("H0: Профессия НЕ связана с решением о подписке на депозит")
print("H1: Профессия связана с решением о подписке на депозит")

ct_j = pd.crosstab(d1['job'], d1['y_bin'])
ct_j_p = pd.crosstab(d1['job'], d1['y_bin'], normalize='index') * 100
print("\nТаблица сопряженности:")
print(ct_j_p.round(1))
print(f"\nВсего наблюдений: {ct_j.sum().sum()}")

c2_j, p_j, dof_j, exp_j = chi2_contingency(ct_j)
v_j = cv(ct_j)
str_j = icv(v_j)
print(f"\nРезультаты теста:")
print(f"X^2 = {c2_j:.2f}, p-value = {p_j:.6f}")
print(f"Cramer's V = {v_j:.4f} ({str_j} связь)")



H₀: Профессия НЕ связана с решением о подписке на депозит
H₁: Профессия связана с решением о подписке на депозит

Таблица сопряженности:
y_bin              0     1
job                       
admin.          78.4  21.6
blue-collar     93.8   6.2
entrepreneur    85.7  14.3
housemaid       90.0  10.0
management      90.0  10.0
retired         61.5  38.5
self-employed   70.0  30.0
services        96.2   3.8
student         80.0  20.0
technician      92.5   7.5
unemployed     100.0   0.0

Всего наблюдений: 297

Результаты теста:
X^2 = 22.24, p-value = 0.013908
Cramer's V = 0.2031 (слабая связь)


Почему выбран χ²-тест:
job и y_bin — категориальные признаки (много категорий против 0/1). Наша задача — проверить, отличается ли распределение y_bin между категориями job. Для проверки зависимости между двумя категориальными переменными используется χ²-тест независимости на таблице сопряженности.

Тест сравнивает наблюдаемые частоты O_{ij} в таблице job × y_bin с ожидаемыми частотами E_{ij}, которые были бы при независимости признаков:

$E_{ij}=\frac{(\text{сумма по строке } i)\cdot(\text{сумма по столбцу } j)}{N}$

Далее вычисляется статистика:
$\chi^2=\sum \frac{(O_{ij}-E_{ij})^2}{E_{ij}}$

# Статистический вывод:
Проверили гипотезу о независимости признаков job и y_bin с помощью χ²-теста независимости.

По результатам теста получили: $\chi^2$ = 22.24, $p-value$ = 0.0139. При уровне значимости $\alpha$ = 0.05 $p-value < \alpha$, поэтому отвергаем нулевую гипотезу. Это означает, что между профессией клиента и фактом подписки на депозит есть статистически значимая связь.

При этом величина эффекта $Cramer’s V$ = 0.2031, что соответствует слабой связи. То есть различия между профессиями существуют, но профессия не является единственным и “самым сильным” фактором подписки.

# Бизнес-вывод:

Профессия клиента действительно влияет на вероятность подписки, поэтому имеет смысл использовать job для сегментации и настройки коммуникации. По таблице долей видно, что конверсия выше у сегментов retired (≈38.5%) и self-employed (≈30%), а ниже — у services (≈3.8%), blue-collar (≈6.2%) и technician (≈7.5%).

Практически это можно применить так:
- Приоритизировать обзвон/касания сегментов с более высокой конверсией (retired, self-employed и т.п.) при ограниченных ресурсах колл-центра.
- Для низкоконверсионных сегментов (services, blue-collar, technician) тестировать другой оффер/скрипт (другая подача, аргументы, условия), а не “лить” на них тот же сценарий.
- Учитывать, что связь слабая (Cramer’s V ≈ 0.20): лучший эффект даст комбинация факторов (job + другие признаки), а не решение только на основе профессии.

# Гипотеза 2
В рамках EDA мы изучили связь признаков с целевой переменной y_bin. Для категориального признака education (уровень образования) мы построили график количество подписок по категориям

На графике «Доля y=1 по уровню образования» видно, что выборочные доли подписки различаются между образовательными группами. Однако такие различия могут быть обусловлены случайными колебаниями выборки. Поэтому мы переходим к формальной проверке статистической значимости этой связи

Так как education — категориальная переменная с несколькими уровнями, а y_bin — бинарная категориальная переменная, корректным способом проверить наличие зависимости является χ²-тест независимости на таблице сопряженности education × y_bin

Если связь реальная, маркетинг может использовать уровень образования как один из факторов для сегментации и персонализации предложений

In [ ]:
print("H0: Уровень образования НЕ связан с решением о подписке на депозит")
print("H1: Уровень образования связан с решением о подписке на депозит")

ct_e = pd.crosstab(d2['education'], d2['y_bin'])
ct_e_p = pd.crosstab(d2['education'], d2['y_bin'], normalize='index') * 100
print("Таблица сопряженности (проценты по строкам):")
print(ct_e_p.round(1))

c2_e, p_e, dof_e, exp_e = chi2_contingency(ct_e)
v_e = cv(ct_e)
str_e = icv(v_e)
print(f"\nРезультаты теста:")
print(f"X^2 = {c2_e:.2f}, p-value = {p_e:.6f}")
print(f"Cramer's V = {v_e:.4f} ({str_e} связь)")

----------------------------------------------------------------------
H₀: Уровень образования НЕ связан с решением о подписке на депозит
H₁: Уровень образования связан с решением о подписке на депозит

Таблица сопряженности (проценты по строкам):
y_bin                   0     1
education                      
basic.4y             89.9  10.1
basic.6y             95.6   4.4
basic.9y             91.3   8.7
high.school          88.1  11.9
professional.course  89.3  10.7
university.degree    84.8  15.2

Результаты теста:
X^2 = 19.85, p-value = 0.001335
Cramer's V = 0.0784 (очень слабая связь)


# Статистический вывод
Проверили гипотезу о независимости признаков education и y_bin с помощью χ²-теста независимости

По результатам теста получили: χ² = 19.85, p-value = 0.0013. При уровне значимости α = 0.05, p-value < α, поэтому отвергаем нулевую гипотезу (H₀). Это означает, что между уровнем образования клиента и фактом подписки на депозит существует статистически значимая связь

Однако величина эффекта, измеренная с помощью Cramer's V, составляет 0.078, что интерпретируется как очень слабая связь. Это указывает на то, что хотя различия между образовательными группами являются не случайными, уровень образования объясняет лишь очень небольшую часть вариации в принятии решения о подписке

# Бизнес-вывод
Статистически значимая связь между образованием и подпиской на депозит обнаружена. Это означает, что образовательные группы ведут себя по-разному.

Клиенты с университетским образованием (15.2% конверсии) демонстрируют наибольшую склонность к подписке. На них можно ориентировать более сложные или долгосрочные финансовые продукты - Сегмент с высокой конверсией

Сегмент с низкой конверсией: Клиенты с образованием basic.6y (4.4% конверсии) имеют крайне низкий отклик на текущее предложение. Для этой группы необходимо тестировать принципиально иные подходы, офферы или каналы коммуникации

Учитывая очень слабую силу связи, не следует принимать решения, опираясь исключительно на уровень образования. Этот признак стоит рассматривать только в комбинации с другими значимыми факторами для построения более точных прогнозных моделей